In [13]:
!pip  install keras numpy pandas scikit-learn keras_preprocessing nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string



In [81]:
!wget https://raw.githubusercontent.com/ImDarkShadow/Sentiment-Analysis-CNN/master/Twitter_Data.csv

--2023-03-10 14:48:29--  https://raw.githubusercontent.com/ImDarkShadow/Sentiment-Analysis-CNN/master/Twitter_Data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20895533 (20M) [text/plain]
Saving to: ‘Twitter_Data.csv.1’

Twitter_Data.csv.1  100%[===================>]  19.93M  --.-KB/s    in 0.09s   

2023-03-10 14:48:30 (215 MB/s) - ‘Twitter_Data.csv.1’ saved [20895533/20895533]



### Read data

In [17]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=0B7XkCwpI5KDYNlNUTTlSS21pQmM' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=0B7XkCwpI5KDYNlNUTTlSS21pQmM" -O GoogleNews-vectors-negative300.bin.gz && rm -rf /tmp/cookies.txt

--2023-03-10 14:30:37--  https://docs.google.com/uc?export=download&confirm=t&id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
Resolving docs.google.com (docs.google.com)... 74.125.135.139, 74.125.135.101, 74.125.135.100, ...
Connecting to docs.google.com (docs.google.com)|74.125.135.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-8s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/4siuheu9ld5qh9fel59b3g9rtulf5goh/1678458600000/06848720943842814915/*/0B7XkCwpI5KDYNlNUTTlSS21pQmM?e=download&uuid=98c0c71c-2eca-41fa-ba3a-8ce70e5ce884 [following]
--2023-03-10 14:30:37--  https://doc-0g-8s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/4siuheu9ld5qh9fel59b3g9rtulf5goh/1678458600000/06848720943842814915/*/0B7XkCwpI5KDYNlNUTTlSS21pQmM?e=download&uuid=98c0c71c-2eca-41fa-ba3a-8ce70e5ce884
Resolving doc-0g-8s-docs.googleusercontent.com (doc-0g-8s-docs.googleusercontent.com)... 142.250.99.132, 2607:f8b0:400e:

In [31]:
data = pd.read_csv('Twitter_Data.csv')
#data = data.drop(data.index[-1])
data = data.dropna()

In [36]:
data.columns = ['Text', 'Label']

In [37]:
data.head()
data.tail()


,Text,Label
162975,why these 456 crores paid neerav modi not reco...,-1.0
162976,dear rss terrorist payal gawar what about modi...,-1.0
162977,did you cover her interaction forum where she ...,0.0
162978,there big project came into india modi dream p...,0.0
162979,have you ever listen about like gurukul where ...,1.0


In [38]:
data.Label.unique()

array([-1.,  0.,  1.])

In [34]:
data.shape

(162969, 2)

In [39]:
pos = []
neg = []
neu = []
for l in data.Label:
    if l == 0:
        pos.append(0)
        neg.append(0)
        neu.append(1)
    elif l == 1:
        pos.append(1)
        neg.append(0)
        neu.append(0)
    elif l == -1:
        pos.append(0)
        neg.append(1)
        neu.append(0)
print(pos)

[0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 

In [24]:
print(neu)

[0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 

In [41]:
data['Pos']= pos
data['Neg']= neg
data['Neu']= neu

In [42]:
data.head()

,Text,Label,Pos,Neg,Neu
0,when modi promised “minimum government maximum...,-1.0,0,1,0
1,talk all the nonsense and continue all the dra...,0.0,0,0,1
2,what did just say vote for modi welcome bjp t...,1.0,1,0,0
3,asking his supporters prefix chowkidar their n...,1.0,1,0,0
4,answer who among these the most powerful world...,1.0,1,0,0


### Clean data

In [43]:
data['Text'] = data['Text'].apply(str)



print(data['Text'].dtype)
print(data['Text'].unique())



object
['when modi promised “minimum government maximum governance” expected him begin the difficult job reforming the state why does take years get justice state should and not business and should exit psus and temples'
 'talk all the nonsense and continue all the drama will vote for modi '
 'what did just say vote for modi  welcome bjp told you rahul the main campaigner for modi think modi should just relax'
 ... 'did you cover her interaction forum where she left '
 'there big project came into india modi dream project but not happened reality'
 'have you ever listen about like gurukul where discipline are maintained even narendra modi rss only maintaining the culture indian more attack politics but someone attack hinduism rss will take action that proud for ']


In [44]:
def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct

data['Text_Clean'] = data['Text'].apply(lambda x: remove_punct(x))

In [45]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [46]:
from nltk import word_tokenize, WordNetLemmatizer
tokens = [word_tokenize(sen) for sen in data.Text_Clean] 

In [47]:
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens] 

In [48]:
def remove_stop_words(tokens): 
    return [word for word in tokens if word not in stoplist]

In [49]:
filtered_words = [remove_stop_words(sen) for sen in lower_tokens] 

In [50]:
result = [' '.join(sen) for sen in filtered_words] 

In [51]:
data['Text_Final'] = result

In [52]:
data['tokens'] = filtered_words

In [53]:
data = data[['Text_Final', 'tokens', 'Label', 'Pos', 'Neg','Neu']]

In [54]:
data[:5]

,Text_Final,tokens,Label,Pos,Neg,Neu
0,modi promised “ minimum government maximum gov...,"[modi, promised, “, minimum, government, maxim...",-1.0,0,1,0
1,talk nonsense continue drama vote modi,"[talk, nonsense, continue, drama, vote, modi]",0.0,0,0,1
2,say vote modi welcome bjp told rahul main camp...,"[say, vote, modi, welcome, bjp, told, rahul, m...",1.0,1,0,0
3,asking supporters prefix chowkidar names modi ...,"[asking, supporters, prefix, chowkidar, names,...",1.0,1,0,0
4,answer among powerful world leader today trump...,"[answer, among, powerful, world, leader, today...",1.0,1,0,0


### Split data into test and train

In [55]:
data_train, data_test = train_test_split(data, test_size=0.10, random_state=42)

In [56]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

2128551 words total, with a vocabulary size of 102056
Max sentence length is 43


In [57]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

236938 words total, with a vocabulary size of 27365
Max sentence length is 43


### Load Google News Word2Vec model

In [58]:
word2vec_path = 'GoogleNews-vectors-negative300.bin.gz'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [59]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

### Get Embeddings

In [60]:
training_embeddings = get_word2vec_embeddings(word2vec, data_train, generate_missing=True)

In [61]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300

### Tokenize and Pad sequences

In [62]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["Text_Final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 102038 unique tokens.


In [63]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [64]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(102039, 300)


In [65]:
test_sequences = tokenizer.texts_to_sequences(data_test["Text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

### Define CNN

In [66]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)

    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [67]:
label_names = ['Pos', 'Neg','Neu']

In [68]:
y_train = data_train[label_names].values

In [69]:
x_train = train_cnn_data
y_tr = y_train

In [70]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 50, 300)      30611700    ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 49, 200)      120200      ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 48, 200)      180200      ['embedding[0][0]']              
                                                                                              

### Train CNN

In [73]:
num_epochs = 5
batch_size = 34

In [74]:
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

Epoch 1/5
3883/3883 [==============================] - 33s 9ms/step - loss: 0.1306 - acc: 0.9288 - val_loss: 0.2389 - val_acc: 0.8811
Epoch 2/5
3883/3883 [==============================] - 35s 9ms/step - loss: 0.1034 - acc: 0.9433 - val_loss: 0.2693 - val_acc: 0.8801
Epoch 3/5
3883/3883 [==============================] - 33s 9ms/step - loss: 0.0846 - acc: 0.9545 - val_loss: 0.3010 - val_acc: 0.8802
Epoch 4/5
3883/3883 [==============================] - 34s 9ms/step - loss: 0.0724 - acc: 0.9617 - val_loss: 0.3136 - val_acc: 0.8814
Epoch 5/5
3883/3883 [==============================] - 34s 9ms/step - loss: 0.0626 - acc: 0.9672 - val_loss: 0.3355 - val_acc: 0.8825


### Test CNN

In [75]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)

16/16 [==============================] - 1s 51ms/step


In [76]:
labels = [1, 0,-1]

In [77]:
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])

In [78]:
sum(data_test.Label==prediction_labels)/len(prediction_labels)

0.41774559734920536

In [79]:
data_test.Label.value_counts()

 1.0    7157
 0.0    5557
-1.0    3583
Name: Label, dtype: int64